# Trematodes analysis - reconstruction of ancestral Trematode and comparison of extant species to the ancestral genome

Run pyham analysis. For pyham to work, we need some basic files that we can get from the omastandalone output: 1) the species tree and 2) the orthoxml file. The species tree gives the relationship between species, and the orthoxml gives all the information about the orthologs, in a hierarchical format.

In [ ]:
# Load in all necessary scientific libraries
import pandas as pd
import pyham
from collections import Counter
from Bio import SeqIO


#  OPTIONAL: only if you want to have the logger information printed
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(name)-12s %(levelname)-8s %(message)s")

# Read in a newick file as a taxonomy reference (species tree), here loaded the tree computed by omastandalone. Each ancestral node has to be labelled.
# Read in a file of all hierarchical orthologous groups computed by omastandalone
# Run ham anaylsis
working_dir = 'your/working/directory'
nwk_file = working_dir + "EstimatedSpeciesTree.nwk"
orthoxml_file =  working_dir + "HierarchicalGroups.orthoxml"
ham_analysis = pyham.Ham(nwk_file, orthoxml_file, use_internal_name=True)



# Set up a dataframe

Now that we have created the main pyham object, we can extract all the information from it. 
1. How many genes and which genes were duplicated, retained or gained from ancestral trematode in all extant trematodes. 
2. How many genes and which genes were duplicated, retained or gained from ancestral trematode in Atriophallophorus winterbourni. 

Pyham deals with ancestral genomes as well as extant genomes. It basically uses the extant genomes and orthology information to reconstruct the ancestral genomes. An ancestral genome can be considered as all the hogs present at a given taxonomic level. In the following analysis, the term "trematode" is used to refer to the ancestral trematode genome.

In this strategy, we will perform a vertical comparison with the trematode ancestral genome to ALL the extant trematode species. Then for each of the species, we can see which genes were duplicated, retained, gained, or lost. Keep in mind that the lost genes don't carry much weight because we are assuming the genomes are fragmented and incomplete. 

We create a dataframe where each row is a gene and the columns will be: gene, species, class (duplicated, retained, etc), hog, hog family size, protein length.

Load in all the protein fasta files and create a dataframe with all the genes per species.

In [ ]:
# Load in all the protein fasta files

genome_filenames = [
"Atriophallophorus_red3.agouti.run1.all.maker.proteins.fa",
"Caenorhabditis_elegans.GCA_000002985.3_WBcel235_protein.fa",
"Clonorchis_sinensis_GCA_003604175.1_ASM360417v1_protein.fa",
"Ecchinococcus_granulosus_GCA_000524195.1_ASM52419v1_protein.fa",
"Echinococcus_multilocularis_GCA_000469725.3_EMULTI002_protein.fa",
"Echinostoma_caproni_GCA_900618425.1_Egypt_0011_upd_protein.fa",
"Fasciola_hepatica_GCA_002763495.2_1.0.allpaths.pg_protein.fa",
"Gyrodactylus_salaris_PRJNA244375.WBPS11_protein_fromwormbase.fa",
"Opisthorchis_felineus_GCA_004794785.1_ICG_Ofel_1.0_protein.fa",
"Opisthorchis_viverrini_GCA_001990785.1_1.0.pg.lrna_protein.fa",
"Pristionchus_pacificus_GCA_000180635.3_El_Paco_protein.fa",
"Schistosoma_bovis_GCA_003958945.1_ASM395894v1_protein.fa",
"Schistosoma_curassoni_GCA_900618015.1_Dakar_0011_upd_protein.fa",
"Schistosoma_haematobium_GCA_000699445.1_1.0_protein.fa",
"Schistosoma_japonicum_GCA_006368765.1_ASM636876v1_protein.fa",
"Schistosoma_mansoni_GCA_000237925.2_ASM23792v2_protein.fa",
"Schistosoma_mattheei_GCA_900617995.1_Denwood_0011_upd_protein.fa",
"Schistostoma_margrebowiei_GCA_900618395.1_Zambia_0011_upd_protein.fa",
"Taenia_solium_PRJNA170813.WBPS11.protein_frompubwormbase.fa",
"Trichobilharzia_regenti_GCA_900618515.1_v1_0_4_001_upd_protein.fa"]

# Create an empty dataframe to input all the data
all_df = pd.DataFrame()

# Read in all the genes from the fasta files
for genome_filename in genome_filenames:
    #read in fasta file
    fasta_file = open(working_dir + "DB/"+ genome_filename)
    #initiate a dictionary to store the lengths of all the genes
    length_dict = {}

    #read each sequence in the fasta file
    for record in SeqIO.parse(fasta_file, 'fasta'):
        name = record.id
        seq = record.seq
        length_dict[name] = len(seq)
    
    #make a dataframe 
    df = pd.DataFrame.from_dict(length_dict, orient="index").reset_index()
    df.rename({"index":"gene", 0:"protein_len"}, inplace=True, axis=1)
    
    #add species
    df['species'] = genome_filename.split(".")[0]
    all_df = pd.concat([all_df, df])
              

# Define the trematodes in the analysis
trematode_genome_filenames = [
"Atriophallophorus_red3.agouti.run1.all.maker.proteins.fa",
"Clonorchis_sinensis_GCA_003604175.1_ASM360417v1_protein.fa",
"Echinostoma_caproni_GCA_900618425.1_Egypt_0011_upd_protein.fa",
"Fasciola_hepatica_GCA_002763495.2_1.0.allpaths.pg_protein.fa",
"Opisthorchis_felineus_GCA_004794785.1_ICG_Ofel_1.0_protein.fa",
"Opisthorchis_viverrini_GCA_001990785.1_1.0.pg.lrna_protein.fa",
"Schistosoma_bovis_GCA_003958945.1_ASM395894v1_protein.fa",
"Schistosoma_curassoni_GCA_900618015.1_Dakar_0011_upd_protein.fa",
"Schistosoma_haematobium_GCA_000699445.1_1.0_protein.fa",
"Schistosoma_japonicum_GCA_006368765.1_ASM636876v1_protein.fa",
"Schistosoma_mansoni_GCA_000237925.2_ASM23792v2_protein.fa",
"Schistosoma_mattheei_GCA_900617995.1_Denwood_0011_upd_protein.fa",
"Schistostoma_margrebowiei_GCA_900618395.1_Zambia_0011_upd_protein.fa",
"Trichobilharzia_regenti_GCA_900618515.1_v1_0_4_001_upd_protein.fa"]


trematode_genomes = [x.split(".")[0] for x in trematode_genome_filenames] 


# Vertical analysis

pyHam allows tracing of HOGs/genes provided in the orthoxml file and through the protein files along each branch of a phylogenetic tree (provided in the newick format) and report the genes from these HOGs that arose through duplication on each branch, got lost on each branch, each appeared on that branch, or were simply retained. The vertical analysis allows for retrieval of all genes and their evolutionary history between the two taxonomic levels, the ancestral and  (i.e. which genes have been duplicated, which genes have been lost, etc).

Define functions: 1. Classify each gene as duplicated, gained, retained and lost with respect to an ancestral genome and define how to run vertical analysis. 2. Convert duplicated, retained and gained genes obtained from comparison of extant to ancestral genome into lists. 3.From gene annotations of Fasciola hepatica, Schistosoma mansoni and Schistosoma japonicum (best studied species) obtain putative gene functions. 

In [ ]:
def get_duplicated_gained_retained_lost_vertical(ham_analysis, genome1, genome2):
    '''performs all vertical comparisons (getting dup, ret, lost, gained genes)
    bettween two genomes (can be ancestral or extant)'''
    genome1 = ham_analysis.get_ancestral_genome_by_name(genome1)
    genome2 = ham_analysis.get_extant_genome_by_name(genome2)
    
    vertical_comparison = ham_analysis.compare_genomes_vertically(genome1, genome2)
    
    retained = vertical_comparison.get_retained()
    duplicated = vertical_comparison.get_duplicated()
    lost = vertical_comparison.get_lost()
    gained = vertical_comparison.get_gained()
    
    return duplicated, gained, retained, lost


def make_list_of_genes(dup_ret_gained):
    '''This function just takes the pyham objects returned from the vertical analysis and converts them to lists'''
    list_of_genes = []
    
    if isinstance(dup_ret_gained, dict):
        for hog, gene in dup_ret_gained.items():
            
            #for duplicated
            if isinstance(gene, list):
                list_of_xrefs = [x.get_dict_xref()['protId'].split(" ")[0] for x in gene]
                list_of_genes.append(list_of_xrefs)

            #for retained
            else:
                xref = gene.get_dict_xref()['protId'].split(" ")[0]
                list_of_genes.append(xref)
    
    #for gained
    if isinstance(dup_ret_gained, list):
        list_of_genes = [x.get_dict_xref()['protId'].split(" ")[0] for x in dup_ret_gained]

    return list_of_genes

well_studied_species = ["Fasciola_hepatica_GCA_002763495", \
                        "Schistosoma_mansoni_GCA_000237925", "Schistosoma_japonicum_GCA_006368765"]

def search_for_putative_function(hog, well_studied_species):
    putative_functions = {}
    hog = ham_analysis.get_hog_by_id(hog)
    genes_in_hog = hog.get_all_descendant_genes_clustered_by_species()
    for species, genes in genes_in_hog.items():
        if species.name in well_studied_species:
            for gene in genes:
                xref = gene.get_dict_xref()
                putative_functions[xref['protId'].split(" ").pop(0)] = xref['protId'],'\n'
    return putative_functions

Run vertical analysis in pyham between each extant trematode and the ancestral trematode genome. 

In [ ]:
for extant_genome in trematode_genomes:
    duplicated, gained, retained, lost = get_duplicated_gained_retained_lost_vertical\
        (ham_analysis, "Trematoda", extant_genome)
    
    # Make lists of genes to facilitate transfer into df
    retained_genes = make_list_of_genes(retained)
    duplicated_genes = make_list_of_genes(duplicated)
    duplicated_genes = [item for sublist in duplicated_genes for item in sublist]
    gained_genes = make_list_of_genes(gained)
    lost_genes = make_list_of_genes(lost)

    # Classify all trematode genes as either duplicatetd, retained, or gained
    all_df.loc[(all_df['gene'].isin(duplicated_genes)),'class'] = 'duplicated'
    all_df.loc[(all_df['gene'].isin(retained_genes)),'class'] = 'retained'
    all_df.loc[(all_df['gene'].isin(gained_genes)),'class'] = 'gained'
    
# Get rid of all other species that are not trematodes from the dataframe 
all_df.loc[(all_df['species'].isin(trematode_genomes)),'trematode'] = 'yes'
all_df.loc[(all_df['species'].isnull()),'trematode'] = 'no'
all_df[:5]
trematode_df = all_df[all_df['trematode']=="yes"]


Define a root HOG (top level HOG) for each gene in the trematode_df table and establish the size of each othese HOGs including genes from all the trematode species. Add that to the table as a column.

In [ ]:
gene_hog_dict = {}

for genome in trematode_df['species'].unique():
    extant_genome = ham_analysis.get_extant_genome_by_name(genome)

    for gene in extant_genome.genes:
        xref = gene.get_dict_xref()['protId'].split(" ")[0]
        hog = gene.get_top_level_hog()
        if hog.is_singleton():
            gene_hog_dict[xref] = "singleton"
        else:
            gene_hog_dict[xref] = gene.get_top_level_hog().hog_id
        
trematode_df['hog'] = trematode_df['gene'].map(gene_hog_dict)

# Number of genes in the whole hog (i.e. the gene family size or hog size, all the species together)

hog_sizes = trematode_df.groupby('hog').size().reset_index()
hog_sizes.rename({0:"family_size"}, inplace=True, axis=1)
trematode_df = pd.merge(left=trematode_df, right=hog_sizes, how="left", on="hog")
trematode_df.loc[(trematode_df['hog']=="singleton"),'family_size'] = 1

# Add the number of genes per hog for each species and call it "extant copy number"

copy_numbers = trematode_df.groupby(['species','hog']).size().reset_index()
copy_numbers.rename({0:"extant_copy_nr"}, inplace=True, axis=1)
trematode_df = pd.merge(left=trematode_df, right=copy_numbers, how="left", on=["hog",'species'])
trematode_df.loc[(trematode_df['hog']=="singleton"),'extant_copy_nr'] = 1


# Genes that arose through duplication since trematode ancestor

In [ ]:
# Get duplicated genes
# Number of genes per HOG that got duplicated in each species since trematode ancestor has to be above or equal to copy_nb_threshold
# Minimum number of species in which genes have to have duplicated in each family above the copy_nb_theshold is defined by species_nb_threshold

copy_nb_threshold = 3
trematode_df[(trematode_df['extant_copy_nr']>=copy_nb_threshold) &\
             (trematode_df['class']=="duplicated")].groupby('species').size()

species_nb_threshold = 11
# Sort the genes into top level HOG and give the HOG name and the number of species present

df = trematode_df[(trematode_df['extant_copy_nr']>=copy_nb_threshold) &\
             (trematode_df['class']=="duplicated")].groupby(['hog','species']).\
size().reset_index(name='count').groupby(['hog']).size().reset_index(name="nb_species")

highly_duplicated_hogs_df = df[df["nb_species"]>=species_nb_threshold]
# Print the putative functions of genes in each HOG given the annotation of well studied species
for hog in highly_duplicated_hogs_df['hog']:
    print("HOG: ", hog)
    functions = search_for_putative_function(hog, well_studied_species), "\n"
    print(functions, "\n")

In [ ]:
# Get highly duplicated genes *specifically* in Atriophallophorus winterbourni

copy_nb_threshold = 3
highly_duplicated_atrio_hogs_df = trematode_df[(trematode_df['species']=="Atriophallophorus_red3") \
             & (trematode_df['class']=="duplicated") \
             & (trematode_df['extant_copy_nr']>=copy_nb_threshold)].groupby("hog").size().reset_index(name="nb_genes")

# Print the putative functions of Atriophallophorus winterbourni genes in each HOG given the annotation of well studied species
for hog in highly_duplicated_atrio_hogs_df['hog']:
    print("HOG: ", hog)
    functions = hog, search_for_putative_function(hog, well_studied_species), "\n"
    print(functions, "\n")
# Print number of genes of A.winterbourni in each HOG
print(len(highly_duplicated_atrio_hogs_df['hog']))    

# Make a list of those genes that arose through duplication in A.winterbourni in each HOG and output them to a file.
highly_duplicated_atrio_genes_df  = trematode_df[(trematode_df['class']=="duplicated") \
                      & (trematode_df['species']=="Atriophallophorus_red3") \
                         & (trematode_df['extant_copy_nr']>copy_nb_threshold)]['gene'].tolist()
with open(working_dir + 'highly_duplicated_atrio_genes.txt', 'w') as filehandle:
    for gene in mygenes_df:
        filehandle.write('%s\n' % gene)
        
# Get protein sequences for all these genes from a concatenated fasta file containing all species proteins all_sequences.fa
for gene in highly_duplicated_atrio_genes_df:
           record_dict = SeqIO.to_dict(SeqIO.parse(working_dir + "DB/" + "all_sequences.fa", "fasta"))
           print(record_dict[gene].seq)   


In [ ]:
# Get highly duplicated for each species just as previously only for A.winterbourni, so it does not have to be shared duplications
# Duplications must have happened anytime after trematode speciation
# copy_nb_threshold - number of duplications since trematode ancestor

copy_nb_threshold = 3
print("AT LEAST " + str(copy_nb_threshold) + " COPIES IN GENOME" + "\n--------------------------\n")

for genome in trematode_genomes:
    print(genome + "\n**********************")
    highly_duplicated_hogs_df = trematode_df[(trematode_df['species']==genome) \
                 & (trematode_df['class']=="duplicated") \
                 & (trematode_df['extant_copy_nr']>copy_nb_threshold)].groupby("hog").size().reset_index(name="nb_genes")

# Print the putative functions of the genes in each HOG given the annotation of well studied species
    for hog in highly_duplicated_hogs_df['hog']:
        print("HOG: ", hog)
        functions = hog, search_for_putative_function(hog, well_studied_species), "\n"
        print(functions, "\n")

# Get protein sequences for all these genes from a concatenated fasta file containing all species proteins all_sequences.fa
    for gene in highly_duplicated_hogs_df:
           record_dict = SeqIO.to_dict(SeqIO.parse(working_dir + "DB/" + "all_sequences.fa", "fasta"))
           print(record_dict[gene].seq)   


# Retained genes since trematode ancestor

In [ ]:
# Get all the retained genes in extant species since the trematode ancestor
# Nb_species_threshold defines how many species minimum the retained genes have to be present in
# Family_size_threshold is the size of the HOG and because we are looking for 1:1 orthologs the family size has to be equal to the number of species present in that family

nb_species_threshold = 14
family_size_threshold = nb_species_threshold

df = trematode_df[(trematode_df['class']=="retained") & (trematode_df['family_size']<=family_size_threshold)].\
    groupby(['hog','species']).size().reset_index(name='count')\
    .groupby(['hog']).size().reset_index(name="nb_species")

highly_conserved_hogs_df = df[df["nb_species"]>= nb_species_threshold]

# Print the putative functions of the genes in each HOG given the annotation of well studied species
for hog in highly_conserved_hogs_df['hog']:
    print("HOG: ", hog)
    functions = hog, search_for_putative_function(hog, well_studied_species), "\n"
    print(functions, "\n")


# Gene Ontology Enrichment Analysis for gained, retained and duplicated genes in each extant species since the ancestral trematode

In order to perform this analysis one needs to obtain all genes from each category per species, not shared genes between all trematodes. One also needs annotation of all genomes used with Pannzer2 (http://ekhidna2.biocenter.helsinki.fi/sanspanz/), OMA (https://omabrowser.org/oma/functions/) and EggNOG (http://eggnog5.embl.de/#/app/home). One also needs to download a gene ontology file describing relationships between GO terms (http://geneontology.org/docs/download-ontology/)

In [ ]:
#Define all necessary functions
import pandas as pd
from goatools import obo_parser
from goatools.go_enrichment import GOEnrichmentStudy
import matplotlib.pyplot as plt
import pyham
from fractions import Fraction

In [ ]:
#load ontologies
#read in downloaded go ontology file (describes terms and relationships among them)
go = obo_parser.GODag(working_dir + 'go.obo')

#You must follow the code above to recontstruct ancestral trematode and create a trematode_df dataframe

In [ ]:
#First make a dataframe for all go terms for all extant genes in whole analysis.
def read_in_pannzera_go_annotations(go_annotations_infile, species):
    '''returns a formatted pandas dataframe with relevant go info'''
    df = pd.read_csv(go_annotations_infile, sep="\t", dtype={'goid': object})
    df = df.rename({"qpid":"gene", "goid":"GO_term"}, axis=1)
    df['GO_term'] = df['GO_term'].apply(lambda x: "GO:"+ x)
    df['source'] = "PANNZER"
    df['species'] = species
    df['gene_go_term_combo'] = df['gene'] + "_" + df['GO_term']
    df = df.dropna(axis=0).drop_duplicates()
    #print("Read in {} GO annotations for {}.".format(len(df), go_annotations_infile))
    df = df[['species','gene', 'GO_term', 'source','gene_go_term_combo']]
    return df

def read_in_oma_go_annotations(go_annotations_infile, species):
    df = pd.read_csv(go_annotations_infile, sep="\t", skiprows=4, header=None)
    df = df[[1,4]].rename({1:"gene", 4:"GO_term"}, axis=1)
    df['source'] = "OMA"
    df['species'] = species
    df['gene_go_term_combo'] = df['gene'] + "_" + df['GO_term']
    df = df.dropna(axis=0).drop_duplicates()
    #print("Read in {} GO annotations for {}.".format(len(df), go_annotations_infile))
    df = df[['species','gene', 'GO_term', 'source','gene_go_term_combo']]
    return df

def read_in_eggnog_go_annotations(go_annotations_infile, species):
    df = pd.read_csv(go_annotations_infile, sep="\t", header=None)
    df = df[[1,2]].rename({1:"gene", 2:"GO_term"}, axis=1)
    df['source'] = "EGGNOG"
    df['species'] = species
    df['gene_go_term_combo'] = df['gene'] + "_" + df['GO_term']
    df = df.dropna(axis=0).drop_duplicates()
    #print("Read in {} GO annotations for {}.".format(len(df), go_annotations_infile))
    df = df[['species','gene', 'GO_term', 'source','gene_go_term_combo']]
    return df

pannzera_go_annotations = ["PAN_GO_Atriophallophorus_red3.txt",
"PAN_GO_Clonorchis_sinensis_GCA_003604175.txt",
"PAN_GO_Ecchinococcus_granulosus_GCA_000524195.txt",
"PAN_GO_Echinococcus_multilocularis_GCA_000469725.txt",
"PAN_GO_Echinostoma_caproni_GCA_900618425.txt",
"PAN_GO_Fasciola_hepatica_GCA_002763495.txt",
"PAN_GO_Opisthorchis_felineus_GCA_004794785.txt",
"PAN_GO_Opisthorchis_viverrini_GCA_001990785.txt",
"PAN_GO_Schistosoma_bovis_GCA_003958945.txt",
"PAN_GO_Schistosoma_curassoni_GCA_900618015.txt",
"PAN_GO_Schistosoma_japonicum_GCA_006368765.txt",
"PAN_GO_Schistosoma_mansoni_GCA_000237925.txt",
"PAN_GO_Schistosoma_mattheei_GCA_900617995.txt",
"PAN_GO_Schistosoma_haematobium_GCA_000699445.txt",
"PAN_GO_Schistostoma_margrebowiei_GCA_900618395.txt",
"PAN_GO_Trichobilharzia_regenti_GCA_900618515.txt"]

oma_go_annotations = ["OMA_GO_Atriophallophorus_red3.txt",
"OMA_GO_Clonorchis_sinensis_GCA_003604175.txt",
"OMA_GO_Echinostoma_caproni_GCA_900618425.txt",
"OMA_GO_Fasciola_hepatica_GCA_002763495.txt",
"OMA_GO_Opisthorchis_viverrini_GCA_001990785.txt",
"OMA_GO_Opisthorchis_felineus_GCA_004794785.txt",
"OMA_GO_Schistosoma_bovis_GCA_003958945.txt",
"OMA_GO_Schistosoma_curassoni_GCA_900618015.txt",
"OMA_GO_Schistosoma_haematobium_GCA_000699445.txt",
"OMA_GO_Schistosoma_japonicum_GCA_006368765.txt",
"OMA_GO_Schistosoma_mansoni_GCA_000237925.txt",
"OMA_GO_Schistosoma_mattheei_GCA_900617995.txt",
"OMA_GO_Schistostoma_margrebowiei_GCA_900618395.txt",
"OMA_GO_Trichobilharzia_regenti_GCA_900618515.txt"]

eggnog_go_annotations = ["EGGNOG_Atriophallophorus_red3.txt",
"EGGNOG_Clonorchis_sinensis_GCA_003604175.txt",
"EGGNOG_Echinostoma_caproni_GCA_900618425.txt",
"EGGNOG_Fasciola_hepatica_GCA_002763495.txt",
"EGGNOG_Opisthorchis_felineus_GCA_004794785.txt",
"EGGNOG_Opisthorchis_viverrini_GCA_001990785.txt",
"EGGNOG_Schistosoma_bovis_GCA_003958945.txt",
"EGGNOG_Schistosoma_curassoni_GCA_900618015.txt",
"EGGNOG_Schistosoma_japonicum_GCA_006368765.txt",
"EGGNOG_Schistosoma_haematobium_GCA_000699445.txt",
"EGGNOG_Schistosoma_mansoni_GCA_000237925.txt",
"EGGNOG_Schistosoma_mattheei_GCA_900617995.txt",
"EGGNOG_Schistostoma_margrebowiei_GCA_900618395.txt",
"EGGNOG_Trichobilharzia_regenti_GCA_900618515.txt"]
#read in all data
all_go_df = pd.DataFrame(columns = ['gene', 'GO_term', 'source', 'species', 'gene_go_term_combo'])

for file in pannzera_go_annotations:
    species = file.split(".")[0].split("PAN_GO_")[1]
    df = read_in_pannzera_go_annotations(working_dir + "/DB/" + file, species)
    all_go_df = all_go_df.append(df)
    
for file in oma_go_annotations:
    species = file.split(".")[0].split("OMA_GO_")[1]
    df = read_in_oma_go_annotations(working_dir + "/DB/" + file, species)
    all_go_df = all_go_df.append(df)
    
for file in eggnog_go_annotations:
    species = file.split(".")[0].split("EGGNOG_")[1]
    df = read_in_eggnog_go_annotations(working_dir + "/DB/" + file, species)
    all_go_df = all_go_df.append(df)    

print(len(all_go_df))

In [ ]:
#Define functions for the whole analysis
species_list= [
"Atriophallophorus_red3",
"Clonorchis_sinensis_GCA_003604175",
"Echinostoma_caproni_GCA_900618425",
"Fasciola_hepatica_GCA_002763495",
"Opisthorchis_felineus_GCA_004794785",
"Opisthorchis_viverrini_GCA_001990785",
"Schistosoma_bovis_GCA_003958945",
"Schistosoma_curassoni_GCA_900618015",
"Schistosoma_haematobium_GCA_000699445",
"Schistosoma_japonicum_GCA_006368765",
"Schistosoma_mattheei_GCA_900617995",
"Schistostoma_margrebowiei_GCA_900618395",
"Trichobilharzia_regenti_GCA_900618515"]

def get_go_associations(choose_source, all_go_df, species):
    '''This function creates the association dictionary, based on if you want the
    intersection, union, one or the other of GO annotations'''
    #initiate an empty dictionary
    associations = {}

    if choose_source == "pannzer":
        df = all_go_df[(all_go_df['source']== "PANNZER") & (all_go_df['species']== species)]
        
    if choose_source == "oma":
        df = all_go_df[(all_go_df['source']== "OMA") & (all_go_df['species']== species)]
        
    if choose_source == "eggnog":
        df = all_go_df[(all_go_df['source']== "EGGNOG") & (all_go_df['species']== species)]
    
        #make a pandas grouped by object to get all the go terms for each gene
    grouped_obj = df.groupby('gene')
        
        #iterate through genes in grouped object
    for gene in grouped_obj.groups.keys():
    
            #create a list of GO terms for each gene
            all_go_terms = grouped_obj.get_group(gene)['GO_term'].tolist()


            #add it to the dictionary
            associations[gene] = set(all_go_terms)
        
    return associations

for species in species_list:
    print("Number of genes with GO terms:", species)
    print("pannzer", len(get_go_associations("pannzer", all_go_df, species)))
    print("oma", len(get_go_associations("oma", all_go_df, species)))
    print("eggnog", len(get_go_associations("eggnog", all_go_df, species)))

def get_population_set(df, species):
    '''Get all the genes in the whole population. This is the background for the GO enrichment.'''
    population = set(df[(df['species']==species)]['gene'].tolist())
    print("There are {} genes in the study population.".format(len(population)))
    return population

def get_study_genes(species, df, gene_class, copy_nb_threshold=1, hog=None):
    '''Returns a dataframe with the specified genes, can add an optional speciific hog'''
    study_genes_df = df[(df['species']==species) & (df['class']==gene_class) & (df['extant_copy_nr']>=copy_nb_threshold)]
    
    if hog!= None:
        study_genes_df = study_genes_df[study_genes_df['hog']==hog]
        
    return study_genes_df

def make_go_study_obj(population, associations, go, propagate_counts=True):
    '''Use goatools to make GO enrichment study object. 
    population == background gene set
    associations == dictionary of gene: {go terms}
    go == go obo file
    '''
    methods = ['bonferroni', 'sidak', 'holm']
    go_study_obj = GOEnrichmentStudy(population, associations, go, 
                              propagate_counts=propagate_counts, 
                              alpha=0.05, 
                              methods=methods)
    return go_study_obj

def run_and_save_goenr(go_study_obj, list_of_genes_for_enrichment, working_dir):
    '''Do the go enrichment and write to file'''
    res = go_study_obj.run_study(list_of_genes_for_enrichment)
    go_study_obj.wr_tsv(working_dir + 'GO_df.tsv', res)
    return res

def read_in_goenr_tsv(enriched_or_depleted, p_cutoff, path_to_tsv):
    '''Read in go enrichment file into a dataframe, choosing if you want to filter for enriched or depleted results'''
    if enriched_or_depleted in ["depleted","p"]:
        e_or_p = "p"
        print("depleted")
    else:
        e_or_p = "e"
        print("enriched")
    df = pd.read_csv(path_to_tsv, sep="\t")
    df = df[(df['enrichment']==e_or_p) & (df['p_bonferroni'] <= p_cutoff)]
    
    return df

def print_for_revigo(go_df, depth_cutoff=0):
    '''simply print the enriched GO terms and their p-values in order to copy and paste into revigo'''
    print(go_df[go_df['depth']>=depth_cutoff][["# GO","p_bonferroni"]].to_string(index=False, justify="left"))
    
def get_fold_change(go_df):
    #convert ratios to floats and calculate fold change
    go_df['ratio_in_study2'] = go_df['ratio_in_study'].apply(lambda x: float(sum(Fraction(s) for s in x.split())))
    go_df['ratio_in_pop2'] = go_df['ratio_in_pop'].apply(lambda x: float(sum(Fraction(s) for s in x.split())))

    go_df['fold_change'] = go_df.apply(lambda x: x['ratio_in_study2']/x['ratio_in_pop2'], axis=1)
    return go_df

def run_go_pipeline_for_genes_part2(study_genes, go_study_obj):
    #second part of GO enrichment (can change the study gene sets)
    #5. run gene ontology enrichment analysis
    run_and_save_goenr(go_study_obj, study_genes, working_dir)

    #6. read in dataframe that was just saved
    go_df = pd.read_csv(working_dir+"GO_df.tsv", sep="\t")

    #7. Filter by p-value and only looking at enriched not depleted go terms!
    go_df = go_df[(go_df['p_bonferroni']<=.05) & (go_df['enrichment']=="e")] 
    
    if len(go_df)>0:
        go_df = get_fold_change(go_df)
        
    return go_df

#Define a function to complete an output GO enrichment table
def make_final_go_df(df, species):
    df = df
    df['species'] = species
    df['class'] = "duplicated"
    df = df[['# GO','NS','depth','name','enrichment', 'fold_change', 'p_uncorrected','p_bonferroni', 'p_holm', 'p_sidak', 'ratio_in_pop', 'ratio_in_pop2','ratio_in_study', 'ratio_in_study2', 'study_count','study_items', 'species', 'class']]
    return df


In [ ]:
#Finally run the analysis
output = pd.DataFrame()

#Change "duplicated" into either "gained" or "retained"
for species in species_list:
    population = get_population_set(trematode_df, species)
    associations = get_go_associations("pannzer", all_go_df, species)
    my_study_genes = trematode_df[(trematode_df['species']== species) & (trematode_df['class']=="duplicated")]['gene'].tolist()
    go_study_obj = make_go_study_obj(population, associations, go)
    run_and_save_goenr(go_study_obj, my_study_genes, working_dir)
    go_df = pd.read_csv(working_dir+"GO_df.tsv", sep="\t")
    go_df = run_go_pipeline_for_genes_part2(my_study_genes, go_study_obj)
    df_final = make_final_go_df(go_df, species)
    if not df_final.empty:
        output = pd.concat([output, df_final])

After you have completed the analysis you might want to concatenate all the data into one dataframe and visualise it. In order to do that one can categorise all the GO terms in GO slim categories (http://geneontology.org/docs/download-ontology/). In our analysis we used the AGR subset. One can also calculate the average information content per GO slim category. For our analysis the IC (Information Content) score was calculated as: IC(t)=−log(p(t)) with p(t) being estimated as the empirical frequency of the term in UniProt-GOA.

In [ ]:
#Load the table created in previous analysis and add the IC values to each GO term
final_go_df = pd.read_csv(working_dir + 'proteinlength/' + 'GO_forallgenes_forallspecies.txt', sep="\t")

ic_df = pd.read_csv(working_dir + 'ic.tsv.gz', sep='\t')
ic_df['GO'] = ic_df['t'].apply(lambda x: "GO:"+ str(x).rjust(7,'0')) #this pads the GO so its in the right format

#add the positive IC to the final_go_df
final_go_df = pd.merge(left=final_go_df, right=ic_df[['GO','pos_ic']], how="left", on="GO")

In [ ]:
#Import necessary functions and AGR GO slim terms
import requests
import json
import numpy as np

slim_AGR = ["GO:0000003",
"GO:0002376",
"GO:0003677",
"GO:0003700",
"GO:0003723",
"GO:0003824",
"GO:0005102",
"GO:0005198",
"GO:0005215",
"GO:0005576",
"GO:0005634",
"GO:0005694",
"GO:0005739",
"GO:0005768",
"GO:0005773",
"GO:0005783",
"GO:0005794",
"GO:0005829",
"GO:0005856",
"GO:0005886",
"GO:0005975",
"GO:0006259",
"GO:0006629",
"GO:0007049",
"GO:0007610",
"GO:0008092",
"GO:0008134",
"GO:0008219",
"GO:0008283",
"GO:0008289",
"GO:0009056",
"GO:0016043",
"GO:0016070",
"GO:0019538",
"GO:0023052",
"GO:0030054",
"GO:0030154",
"GO:0030234",
"GO:0030246",
"GO:0031410",
"GO:0032502",
"GO:0032991",
"GO:0036094",
"GO:0038023",
"GO:0042592",
"GO:0042995",
"GO:0045202",
"GO:0046872",
"GO:0050877",
"GO:0050896",
"GO:0051234",
"GO:0097367",
"GO:1901135"]

In [ ]:
#These are the terms from the df that I want to map to their GO slim term
go_terms_to_slim = list(set(final_go_df['GO'].tolist()))
len(go_terms_to_slim)

In [ ]:
#Use the EBI website to map go terms to their slim term

def format_go_terms_for_url(list_of_go_terms):
    '''Prepares a list of go terms for url by concatenting them in the special format'''
    go_ids_just_numbers = ["GO%3A" + x.split(":")[1] + '%2C' for x in list_of_go_terms]
    go_ids_for_url = ''.join(go_ids_just_numbers) 
    return(go_ids_for_url)

def get_slim_df(slim_set, go_terms_to_slim):
    '''makes the url and request to map the go terms to their go slim term. 
    Stores result in a df'''
    
    slim_url = "https://www.ebi.ac.uk/QuickGO/services/ontology/go/slim?slimsToIds=" + format_go_terms_for_url(slim_set)
    
    go_ids_url = format_go_terms_for_url(go_terms_to_slim)

    requestURL = slim_url + "&slimsFromIds=" + go_ids_url + "&relations=is_a%2Cpart_of%2Coccurs_in%2Cregulates"

    r = requests.get(requestURL, headers={ "Accept" : "application/json"})

    if not r.ok:
      r.raise_for_status()
      sys.exit()

    slim_df = pd.DataFrame(r.json()['results'])
    slim_df['slimsToIds'] = slim_df['slimsToIds'].apply(lambda x: x[0])

    return slim_df

#split terms to map to go slim into chunks
chunked_go_terms_to_slim = np.array_split(go_terms_to_slim, 10)

slim_df = pd.DataFrame()

#run the requests by chunks
for chunk in chunked_go_terms_to_slim:
    tmp_df = get_slim_df(slim_AGR, chunk)
    slim_df = slim_df.append(tmp_df)
    
slim_df[:5]

In [ ]:
#But we don't know the names of the GO slim terms, so we can also get that from the EBI API:
def get_slim_names(go_ids_to_get_names_for):
    '''This function just gets the name of a given GO term'''
    
    requestURL_names = "https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/" + format_go_terms_for_url(go_ids_to_get_names_for)

    r = requests.get(requestURL_names, headers={ "Accept" : "application/json"})

    if not r.ok:
      r.raise_for_status()
      sys.exit()

    responses = r.json()['results']
    names_df = pd.DataFrame(responses)
    names_df = names_df[['id','name']]
    names_df = names_df.rename({'name':"go_slim_name"}, axis=1)
    
    return names_df


#split into chunks so server will run
chunked_go_terms = np.array_split(slim_df['slimsToIds'], 10)

names_df = pd.DataFrame()

for chunk in chunked_go_terms:
    tmp_df = get_slim_names(chunk)
    names_df = names_df.append(tmp_df)
    
names_df = names_df.drop_duplicates()
    
names_df[:5]

In [ ]:
#Create the final GO table
final_go_df = pd.merge(left=final_go_df, right=slim_df, how="left", left_on="GO", right_on="slimsFromId")
final_go_df = pd.merge(left=final_go_df, right=names_df, left_on="slimsToIds", right_on="id")

final_go_df = final_go_df[[ 'species','gene class','GO','name','id','go_slim_name','NS', 'depth', 'enrichment',
         'p_uncorrected','p_bonferroni', 'p_holm', 'p_sidak','ratio_in_pop', 'ratio_in_pop2',
       'ratio_in_study', 'ratio_in_study2',  'fold_change', 'study_count','study_items', 'pos_ic']]
final_go_df = final_go_df.rename({"id":"GO_slim"}, axis=1)


In [ ]:
import re

def split_study_genes(study_items_string_list, species_of_interest):   
    '''This function takes a series of study genes (strings) from the GO enrichment and separates them all in a list'''
    
    
    #initiate empty list where will will store the final list of genes
    mygenes = []

    #input data
    #print("This is what the original data looks like: ")

    #go through all the cells of study items, if there are multiple rows (i.e. multiple go terms for a given go slim)
    for study_items in study_items_string_list:
        #how many genes?
        #print("nb genes: ", len(study_items))

        #what is the datatype? (i.e. string vs. list)
        #print(type(study_items))
        #print("study_items: ", study_items, "\n")

        #split each element (study_item) into individual genes
        separate_genes = study_items.split(",")
        #print("separate_genes: ", separate_genes,"\n")

        #This part removes the square brackets from ancestral genes, as well as whitespace (strip)
        separate_genes = [x.replace("[", "") for x in separate_genes]
        separate_genes = [x.replace("]", "") for x in separate_genes]
        separate_genes = [x.strip() for x in separate_genes]

        #print("separate_genes, with whitespace and brackets removed: ", separate_genes, "\n")
        #print("separate_genes is now a {}.\n".format(type(separate_genes)))

        #add our list of genes to mygenes
        mygenes.append(separate_genes)

    #print("---------------ALL LOOPS DONE---------------\n")

    #what does mygenes look like?
    #print("mygenes: ", mygenes, "\n")

    #we can see that mygenes is a list of lists, so we need to flatten the list
    mygenes = [item for sublist in mygenes for item in sublist]

    #now we can see it is one big list, not several lists anymore
    #print("mygenes, only 1 list:", mygenes, "\n")

    #now we have to remove duplicates from the list
    mygenes = set(mygenes)
    #print("final mygenes: ", mygenes, "\n")

    return mygenes

In [ ]:
go_slim_names = set(final_go_df['go_slim_name'].tolist())


#initiate and emtpy data frame to store the final counts
go_slim_names_gene_counts_df = pd.DataFrame()

#loop through all of species, gene classes, and go slim names
for specie_of_interest in species_of_interest:
    for gene_class in gene_classes:
        for go_slim_name in go_slim_names:
            
            #select the data we want, which are the rows that correspond to our ancestor, gene class, and go slim
            subset_df = final_go_df[(final_go_df['species']== specie_of_interest) &\
                                    (final_go_df['gene class']== gene_class) &\
                                       (final_go_df['go_slim_name']==go_slim_name)]
            
            #we only want study items (i.e. enriched genes)
            study_items = subset_df['study_items']
            
            #get the mean positive information content
            mean_pos_ic = subset_df['pos_ic'].mean()
            
            #use the function from the code block above to split the study genes
            mygenes = split_study_genes(study_items, specie_of_interest)
            
            #how many are there?
            count = len(mygenes)
            
            #make a row containing all of this information...
            tmp_df = pd.DataFrame({"species": [specie_of_interest], "gene class": [gene_class],\
                                   "go_slim_name": [go_slim_name], "nb_genes": count, "mean_IC": mean_pos_ic})
            
            #and add it to the 
            go_slim_names_gene_counts_df = go_slim_names_gene_counts_df.append(tmp_df)
            

#There will be some blanks in the dataframe because not every ancestor or gene class has enriched genes in every GO slim category. So we fill those with 0.
go_slim_names_gene_counts_df = go_slim_names_gene_counts_df.fillna(0)

In [ ]:
#Output
final_go_df.to_csv(working_dir + "proteinlength/" + "final_go_df.txt", sep="\t")
go_slim_names_gene_counts_df.to_csv(working_dir + "proteinlength/" + "go_slim_names_gene_counts_df.txt", sep="\t")

In [ ]:
#Visualise
import seaborn as sns 


for gene_class in gene_classes:
    annotations = go_slim_names_gene_counts_df[go_slim_names_gene_counts_df['gene class']==gene_class]\
                .pivot("species","go_slim_name","nb_genes").T[[
"Atriophallophorus_red3",
"Clonorchis_sinensis_GCA_003604175",
"Echinostoma_caproni_GCA_900618425",
"Fasciola_hepatica_GCA_002763495",
"Opisthorchis_felineus_GCA_004794785",
"Opisthorchis_viverrini_GCA_001990785",
"Schistosoma_bovis_GCA_003958945",
"Schistosoma_curassoni_GCA_900618015",
"Schistosoma_haematobium_GCA_000699445",
"Schistosoma_japonicum_GCA_006368765",
"Schistosoma_mattheei_GCA_900617995",
"Schistostoma_margrebowiei_GCA_900618395",
"Trichobilharzia_regenti_GCA_900618515"]]


    fig, ax = plt.subplots(figsize=(20,20))

    sns.heatmap(go_slim_names_gene_counts_df[go_slim_names_gene_counts_df['gene class']==gene_class]\
                .pivot("species","go_slim_name","mean_IC").T[[
"Atriophallophorus_red3",
"Clonorchis_sinensis_GCA_003604175",
"Echinostoma_caproni_GCA_900618425",
"Fasciola_hepatica_GCA_002763495",
"Opisthorchis_felineus_GCA_004794785",
"Opisthorchis_viverrini_GCA_001990785",
"Schistosoma_bovis_GCA_003958945",
"Schistosoma_curassoni_GCA_900618015",
"Schistosoma_haematobium_GCA_000699445",
"Schistosoma_japonicum_GCA_006368765",
"Schistosoma_mattheei_GCA_900617995",
"Schistostoma_margrebowiei_GCA_900618395",
"Trichobilharzia_regenti_GCA_900618515"]],\
                vmin=0, cmap="YlGnBu",linewidths=.5,\
                annot=annotations, annot_kws={"size": 13}, fmt='g')
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    plt.title(gene_class, fontsize=20)